In [1]:
from fenics import *
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
set_log_level(30)
import random
from dolfin import interpolate, Expression, FunctionSpace, Constant, UserExpression
from dotenv import load_dotenv
import os
load_dotenv()
import numpy as np
import math

In [2]:
nueva_ruta = '/media/erick/TOSHIBA EXT/results_06032025/strong_allee/mu_1'

In [3]:
save_images = os.getenv('SAVE_IMAGES')

# Define constants alpha and beta
D_c = float(os.getenv('D_c'))
D_s = float(os.getenv('D_s'))
D_i = float(os.getenv('D_i'))

rc = float(os.getenv('rc'))
rs = float(os.getenv('rs'))
rd = float(os.getenv('rd'))

alpha = float(os.getenv('alpha'))
delta = float(os.getenv('delta'))
beta = float(os.getenv('beta'))
alle = float(os.getenv('alle'))
gamma = float(os.getenv('gamma'))
eta = float(os.getenv('eta'))
mu = float(os.getenv('mu'))

# Define time parameters
T = float(os.getenv('T'))
dt = float(os.getenv('dt'))
nb = int(os.getenv('nb'))

# Define mesh and space
nodes_in_xaxis = int(os.getenv('nodes_in_xaxis'))
nodes_in_yaxis = int(os.getenv('nodes_in_yaxis'))
space_size = int(os.getenv('space_size'))


# Condición inicial en donde el cancer se origina en puntos
centers = [(5, 5), (15, 15)]  # Coordenadas donde el cáncer se origina
radius = 1.0  # Radio alrededor de los centros
value_outside = 0.0  # Valor fuera de los radios
value_inside = 0.01  # Valor dentro de los radios

# Cambiar al nuevo directorio
os.chdir(nueva_ruta)

In [4]:
# Condición inicial aleatoria
class RandomExpression(UserExpression):
    def __init__(self, min_val, max_val, **kwargs):
        super().__init__(**kwargs)
        self.min_val = min_val
        self.max_val = max_val

    def eval(self, value, x):
        value[0] = random.uniform(self.min_val, self.max_val)

In [5]:
# Definir condiciones iniciales
class CancerInitialCondition(UserExpression):
    def __init__(self, centers, radius, value_outside, value_inside, **kwargs):
        super().__init__(**kwargs)
        self.centers = centers
        self.radius = radius
        self.value_outside = value_outside
        self.value_inside = value_inside

    def eval(self, value, x):
        value[0] = self.value_outside
        for center in self.centers:
            distance = math.sqrt((x[0] - center[0])**2 + (x[1] - center[1])**2)
            if distance <= self.radius:
                value[0] = self.value_inside
                break

In [6]:
def NonlinearSolver(F, field):
    J = derivative(F, field)
    problem = NonlinearVariationalProblem(F, field, bcs=[], J=J)
    solver = NonlinearVariationalSolver(problem)

    # Configuración base del solver
    prm = solver.parameters["snes_solver"]
    prm["method"] = "vinewtonrsls"
    prm["maximum_iterations"] = 10000
    prm["relative_tolerance"] = 1e-6
    prm["absolute_tolerance"] = 1e-8
    prm["report"] = True
    prm["error_on_nonconvergence"] = False

    # Lista de métodos a probar
    linear_solver_options = ["mumps", "superlu", "gmres", "bicgstab", "cg"]
    preconditioner_options = ["none", "amg", "ilu", "icc"]

    # Intenta con la mejor opción primero
    try:
        prm["linear_solver"] = "mumps"
        prm["preconditioner"] = "none"
        solver.solve()
        return solver
    except RuntimeError as e:
        print(f"⚠️ Error con mumps: {e}. Probando otras opciones...")

    # Si falla, probar combinaciones de métodos y precondicionadores
    for linear_solver in linear_solver_options:
        for preconditioner in preconditioner_options:
            try:
                prm["linear_solver"] = linear_solver
                prm["preconditioner"] = preconditioner
                print(f"🔄 Probando con solver={linear_solver}, precondicionador={preconditioner}...")
                solver.solve()
                print(f"✅ Convergió con solver={linear_solver}, precondicionador={preconditioner}")
                return solver
            except (RuntimeError, ConvergenceError) as e:
                print(f"❌ Falló con solver={linear_solver}, precondicionador={preconditioner}: {e}")
                continue

    # Si no converge con ninguna combinación
    raise RuntimeError("No se pudo resolver el sistema no lineal con ninguno de los métodos probados.")


In [7]:
# Función para convertir campos de FEniCS a numpy
def field_to_numpy_array(fenics_field, space_size, step, field_name, nb, sample_rate=0.1): 
    sample_points = np.arange(0, space_size + sample_rate, sample_rate)
    field_array = np.empty((len(sample_points), len(sample_points)), dtype=float)

    for i, val_x in enumerate(sample_points):
        for j, val_y in enumerate(sample_points):
            try:
                valor = fenics_field(val_x, val_y)
            except:
                valor = 0  
            field_array[i, j] = valor

    filename = f"matrix_{field_name}_{step}_nb_{nb}.txt"
    np.savetxt(filename, field_array, delimiter="\t")
    return None

In [8]:
def plot_fields(field_1, field_2, field_3, block):
    plt.figure(figsize=(15, 8))

    plt.subplot(1, 3, 1)
    p1 = plot(field_1)
    p1.set_cmap("seismic")
    plt.title(f'Solution for c at time {t:.3f}')
    aspect = 20
    pad_fraction = 0.5
    ax = plt.gca()
    divider = make_axes_locatable(ax)
    width = axes_size.AxesY(ax, aspect=1./aspect)
    pad = axes_size.Fraction(pad_fraction, width)
    cax = divider.append_axes("right", size=width, pad=pad)
    plt.colorbar(p1, cax=cax)


    plt.subplot(1, 3, 2)
    p2 = plot(field_2)
    p2.set_cmap("gray")
    plt.title(f'Solution for s at time {t:.3f}')
    aspect = 20
    pad_fraction = 0.5
    ax = plt.gca()
    divider = make_axes_locatable(ax)
    width = axes_size.AxesY(ax, aspect=1./aspect)
    pad = axes_size.Fraction(pad_fraction, width)
    cax = divider.append_axes("right", size=width, pad=pad)
    plt.colorbar(p2, cax=cax)
    
    plt.subplot(1, 3, 3)
    p3 = plot(field_3)
    p3.set_cmap("viridis")
    plt.title(f'Solution for i at time {t:.3f}')
    aspect = 20
    pad_fraction = 0.5
    ax = plt.gca()
    divider = make_axes_locatable(ax)
    width = axes_size.AxesY(ax, aspect=1./aspect)
    pad = axes_size.Fraction(pad_fraction, width)
    cax = divider.append_axes("right", size=width, pad=pad)
    plt.colorbar(p3, cax=cax)

    plt.tight_layout(pad=4)
    if save_images == 'Y':
        plt.savefig(f'fields_block_{block}._step_{t:.3f}.png')
        plt.show()
        pass
    else:
        plt.show()

In [9]:
# Espacio de funciones
def create_space_function(space_size, nodes_in_xaxis, nodes_in_yaxis):
    p0 = Point(0.0, 0.0)
    p1 = Point(space_size, space_size)
    mesh = RectangleMesh(p0, p1, nodes_in_xaxis, nodes_in_yaxis, "right/left")
    V = FunctionSpace(mesh, 'P', 1)
    return V

In [10]:
# Función de inicialización con valores más estables
class StableInitialCondition(UserExpression):
    def __init__(self, min_val, max_val, **kwargs):
        super().__init__(**kwargs)
        self.min_val = min_val
        self.max_val = max_val

    def eval(self, value, x):
        value[0] = 0.5 * (self.min_val + self.max_val)  # Valor estable

In [11]:
def solve_dynamics():

    # Define functions for c, s, and i
    V = create_space_function(space_size, nodes_in_xaxis, nodes_in_yaxis)

    c = Function(V)
    s = Function(V)
    i = Function(V)

    phi_c = TestFunction(V)
    phi_s = TestFunction(V)
    phi_i = TestFunction(V)

    # # Define the weak forms for the equations with time dependence

    # Condición inicial aleatoria   
    c_n = interpolate(RandomExpression(min_val=0, max_val=0.02, degree=2), V)
    # Condición inicial con concentración definida 
    # c_n = interpolate(CancerInitialCondition(centers, radius, value_outside, value_inside, degree=2), V)
    s_n = interpolate(RandomExpression(min_val=0, max_val=0.2, degree=2), V)
    i_n = interpolate(RandomExpression(min_val=0, max_val=0.2, degree=2), V)
    
    if mu == 0:
        F_c = ((c - c_n) / dt) * phi_c * dx + D_c * dot(grad(c), grad(phi_c)) * dx + rc * c * (1 - c) * ((c - alle) / (1 - alle)) * phi_c * dx - c*(alpha*s**2 + beta*i**2) * phi_c * dx
        F_s = ((s - s_n) / dt) * phi_s * dx + D_s * dot(grad(s), grad(phi_s)) * dx + rs * s * (1 - s) * phi_s * dx - gamma * c**2 * s * phi_s * dx + delta * i**2 * s * phi_s * dx
        F_i = ((i - i_n) / dt) * phi_i * dx + D_i * dot(grad(i), grad(phi_i)) * dx + rd * i * (1 - i) * phi_i * dx + delta * i * s**2 * phi_i * dx - c**2 * i * eta * phi_i * dx
    else:
        F_c = ((c - c_n) / dt) * phi_c * dx + D_c * dot(grad(c), grad(phi_c)) * dx + rc * c * (1 - c) * ((c - alle) / (1 - alle)) * phi_c * dx - c*(alpha*s**2 + beta*i**2) * phi_c * dx - mu * c*(gamma * s**2 + eta*i**2) * phi_c * dx
        F_s = ((s - s_n) / dt) * phi_s * dx + D_s * dot(grad(s), grad(phi_s)) * dx + rs * s * (1 - s) * phi_s * dx - gamma * c**2 * s * phi_s * dx + delta * i**2 * s * phi_s * dx - ((s*c**2*alpha*mu)/2) * phi_s * dx
        F_i = ((i - i_n) / dt) * phi_i * dx + D_i * dot(grad(i), grad(phi_i)) * dx + rd * i * (1 - i) * phi_i * dx + delta * i * s**2 * phi_i * dx - c**2 * i * eta * phi_i * dx - ((i*c**2*beta*mu)/2) * phi_i * dx

    
    solver_c = NonlinearSolver(F_c, c)
    solver_s = NonlinearSolver(F_s, s)
    solver_i = NonlinearSolver(F_i, i)

    return solver_c, solver_s, solver_i, c, s, i, c_n, s_n, i_n, dx

In [12]:
# Iteración en bloques
for block in range(1, nb + 1):
    t = 0
    solver_c, solver_s, solver_i, c, s, i, c_n, s_n, i_n, dx = solve_dynamics()
    while t < T:
        print(f'Bloque {block}, Tiempo {t:.3f}')

        # Resolver el sistema
        try:
            solver_c.solve()
            solver_s.solve()
            solver_i.solve()
        except RuntimeError as e:
            print(f"Error en bloque {block} en t={t:.3f}: {e}")
            break

        # Actualizar solución previa
        c_n.assign(c)
        s_n.assign(s)
        i_n.assign(i)

        field_to_numpy_array(c, space_size, "{:.3f}".format(t), "c", block)
        field_to_numpy_array(s, space_size, "{:.3f}".format(t), "s", block)
        field_to_numpy_array(i, space_size, "{:.3f}".format(t), "i", block)
        if block == 1:
            # plot_fields(c_n, s_n, i_n, block)
            t += dt
        else:
            t += dt

Bloque 1, Tiempo 0.000
Bloque 1, Tiempo 0.005
Bloque 1, Tiempo 0.010
Bloque 1, Tiempo 0.015
Bloque 1, Tiempo 0.020
Bloque 1, Tiempo 0.025
Bloque 1, Tiempo 0.030
Bloque 1, Tiempo 0.035
Bloque 1, Tiempo 0.040
Bloque 1, Tiempo 0.045
Bloque 1, Tiempo 0.050
Bloque 1, Tiempo 0.055
Bloque 1, Tiempo 0.060
Bloque 1, Tiempo 0.065
Bloque 1, Tiempo 0.070
Bloque 1, Tiempo 0.075
Bloque 1, Tiempo 0.080
Bloque 1, Tiempo 0.085
Bloque 1, Tiempo 0.090
Bloque 1, Tiempo 0.095
Bloque 1, Tiempo 0.100
Bloque 1, Tiempo 0.105
Bloque 1, Tiempo 0.110
Bloque 1, Tiempo 0.115
Bloque 1, Tiempo 0.120
Bloque 1, Tiempo 0.125
Bloque 1, Tiempo 0.130
Bloque 1, Tiempo 0.135
Bloque 1, Tiempo 0.140
Bloque 1, Tiempo 0.145
Bloque 1, Tiempo 0.150
Bloque 1, Tiempo 0.155
Bloque 1, Tiempo 0.160
Bloque 1, Tiempo 0.165
Bloque 1, Tiempo 0.170
Bloque 1, Tiempo 0.175
Bloque 1, Tiempo 0.180
Bloque 1, Tiempo 0.185
Bloque 1, Tiempo 0.190
Bloque 1, Tiempo 0.195
Bloque 1, Tiempo 0.200
Bloque 1, Tiempo 0.205
Bloque 1, Tiempo 0.210
Bloque 1, T